Import libraries

In [296]:
import gmaps
import pandas as pd
import numpy as np
import time
import ipywidgets as widgets
import threading

from ipywidgets import interact, interact_manual
from IPython.display import display, clear_output
from datetime import datetime
from datetime import timedelta
from matplotlib import pyplot as plt

**Authenticate gmaps**

In [49]:
apikey = open('apikey.txt', 'r').read()

gmaps.configure(api_key=apikey)

**Read data**

In [42]:
filename = 'orders_autumn_2020.csv'

order_data = pd.read_csv(filename)

**Helper functions**

In [8]:
def filter_data(data, hours, current_time):
    """
    Return datapoints which fall within certain limits given the current time and the limit resolution.

    Parameters
    ----------
    data : pandas.DataFrame
        Input data to be filtered.
    current_time : DateTime
        format yyyy-MM-dd HH:mm:ss.SSS
    hours: float
        resolution in hours.

    Returns
    -------
    filtered_data : pandas.DataFrame
        Data that is filtered based on resolution around current timestamp.

    """
    time_difference = order_data['TIMESTAMP'].map(string_to_date) - current_time
    upper_limit = timedelta(hours=hours)
    lower_limit = timedelta(hours=-1*hours)
    
    
    filtered_data = data[(lower_limit < time_difference) & (time_difference < upper_limit)]

    return filtered_data

**Plot map**

**Visualize orders w.r.t time**

In [343]:
class Heatmap(object):
    
    def __init__(self, data, step_size, sliding_window_size, frame_delay=1):
        self._data = data
        self._locations = data[["VENUE_LAT", "VENUE_LONG"]]
        
        self._sliding_window_size = sliding_window_size
        self._step_size = step_size
        self._frame_delay = frame_delay
        self._running = False
        
        self._figure = gmaps.figure()
        self._current_index = 0
        self._heatmap = gmaps.heatmap_layer(self._locations[self._current_index : self._sliding_window_size])
        self._figure.add_layer(self._heatmap)
        
        
    
    ##### 
    # Public functions
    #####
    def start_interactive_mode(self):
        self._init_interactives()
        
    def set_frame_delay(self, frame_delay):
        self._frame_delay = frame_delay
        
    def render(self):
        return display(self._figure)
    
    
    #####
    # Interactive elements
    #####
    def _init_interactives(self):
        self._init_date_pickers()
        self._init_point_size_slider()
        self._init_start_button()
        self._init_stop_button()
        self._init_output_panel()
        
    
    def _init_date_pickers(self):
        interact(self._set_interactive_date_limits,
            start_date = widgets.DatePicker(value = pd.to_datetime('2020-08-10')),
            end_date = widgets.DatePicker(value = pd.to_datetime('2020-08-12'))
        )
    
    
    def _init_point_size_slider(self):
        interact(self._set_pointsize,
            pointsize = widgets.IntSlider(value=50, continous_update=False)
        )
        
        
    def _init_start_button(self):
        im = interact_manual(self._start_animation, description = "Start animation")
        im.widget.children[0].description = "Start animation"
        
        
        
    def _init_stop_button(self):
        im = interact_manual(self._stop_animation)
        im.widget.children[0].description = "Stop animation"
    
    
    def _init_output_panel(self):
        self._out = widgets.Output(layout={'border': '1px solid black'})
        self._out.layout.width = 'calc(50%)'
        self._out.layout.height = '8ex'
        display(self._out)
        
    
    #####
    # Heatmap accessor functions
    ####
    def _start_animation(self):
        self._running = True
        
        start_index = self._find_closest_entry(pd.to_datetime(self._current_start_date))
        end_index = self._find_closest_entry(pd.to_datetime(self._current_end_date))
        
        self._current_index = start_index
        
        def run():
            while self._current_index < (end_index - self._step_size - self._sliding_window_size) and self._running:
                self._current_index = (self._current_index + self._step_size)

                self._render_data(self._current_index, self._current_index + self._sliding_window_size)
                time.sleep(self._frame_delay)
        
            self._running = False
            display("")
        
        thread = threading.Thread(target=run)
        thread.start()
    
    
    def _stop_animation(self):
        self._running = False
    
    
    def _find_closest_entry(self, date_to_search):
        return pd.to_datetime(self._data["TIMESTAMP"]).sub(date_to_search).abs().idxmin()
    
    
    def _set_pointsize(self, pointsize):
        self._heatmap.point_radius = pointsize
    

    def _set_interactive_date_limits(self, start_date, end_date):
        # Update currently chosen dates, so hourly slider works
        self._current_start_date = start_date
        self._current_end_date = end_date
        
        start_index = self._find_closest_entry(pd.to_datetime(start_date))
        end_index = self._find_closest_entry(pd.to_datetime(end_date))
        self._render_data(start_index, end_index)
    
    
    # update the locations drawn on the heatmap
    def _render_data(self, start_index, end_index):
        if self._running:
            with self._out:
                clear_output()
                output_text = print("Date interval displayed: \n{0} - {1}".format(self._data.iloc[start_index]["TIMESTAMP"],
                                               self._data.iloc[end_index]["TIMESTAMP"]))
                display(output_text)
        self._heatmap.locations = self._locations[start_index : end_index] 
        

In [349]:
# How many orders are we stepping forward on each iteration
step_size = 4
# How many orders are we including in single view
sliding_window_size = 40
# How long do you want to display single frame? (sec)
frame_delay = 0.3

heatmap = Heatmap(order_data, step_size, sliding_window_size, frame_delay)

In [350]:
heatmap.start_interactive_mode()
heatmap.render()

interactive(children=(DatePicker(value=Timestamp('2020-08-10 00:00:00'), description='start_date'), DatePicker…

interactive(children=(IntSlider(value=50, description='pointsize'), Output()), _dom_classes=('widget-interact'…

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

Output(layout=Layout(border='1px solid black', height='8ex', width='calc(50%)'))

Figure(layout=FigureLayout(height='420px'))

''